In [1]:
import pandas as pd
import nltk

In [2]:
sentiment_data = pd.read_csv('/content/sentiment_analysis.csv')

In [3]:
sentiment_data.head(10)

tweet_id  ...               user_timezone
0  570306133677760513  ...  Eastern Time (US & Canada)
1  570301130888122368  ...  Pacific Time (US & Canada)
2  570301083672813571  ...  Central Time (US & Canada)
3  570301031407624196  ...  Pacific Time (US & Canada)
4  570300817074462722  ...  Pacific Time (US & Canada)
5  570300767074181121  ...  Pacific Time (US & Canada)
6  570300616901320704  ...  Pacific Time (US & Canada)
7  570300248553349120  ...  Pacific Time (US & Canada)
8  570299953286942721  ...  Pacific Time (US & Canada)
9  570295459631263746  ...  Eastern Time (US & Canada)

[10 rows x 15 columns]

In [4]:
sentiment_data.shape

(14640, 15)

In [5]:
sentiment_df = sentiment_data.drop(sentiment_data[sentiment_data['airline_sentiment_confidence']<0.5].index, axis= 0)

In [6]:
sentiment_df.shape

(14404, 15)

In [7]:
X = sentiment_df['text']
Y = sentiment_df['airline_sentiment']

In [8]:
# Cleaning our text data:

from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
stop_words = stopwords.words('english')
punctuations = string.punctuation

In [11]:
import re
nltk.download('wordnet')

clean_data = []
for i in range(len(X)):
  text = re.sub('[^a-zA-Z]', ' ',X.iloc[i])
  text = text.lower().split()
  text = [lemmatizer.lemmatize(word) for word in text if (word not in stop_words) and (word not in punctuations)]
  text = ' '.join(text)
  clean_data.append(text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
clean_data

['virginamerica dhepburn said',
 'virginamerica today must mean need take another trip',
 'virginamerica really aggressive blast obnoxious entertainment guest face amp little recourse',
 'virginamerica really big bad thing',
 'virginamerica seriously would pay flight seat playing really bad thing flying va',
 'virginamerica yes nearly every time fly vx ear worm go away',
 'virginamerica really missed prime opportunity men without hat parody http co mwpg grezp',
 'virginamerica well',
 'virginamerica amazing arrived hour early good',
 'virginamerica know suicide second leading cause death among teen',
 'virginamerica lt pretty graphic much better minimal iconography',
 'virginamerica great deal already thinking nd trip australia amp even gone st trip yet p',
 'virginamerica virginmedia flying fabulous seductive sky u take stress away travel http co ahlxhhkiyn',
 'virginamerica thanks',
 'virginamerica sfo pdx schedule still mia',
 'virginamerica excited first cross country flight lax mc

In [13]:
Y

0         neutral
2         neutral
3        negative
4        negative
5        negative
           ...   
14634    negative
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14404, dtype: object

In [14]:
# defining Sentiments:

sentiments = ['negative' , 'neutral', 'positive']
Y = Y.apply(lambda x: sentiments.index(x))

In [15]:
Y.head()

0    1
2    1
3    0
4    0
5    0
Name: airline_sentiment, dtype: int64

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features = 5000, stop_words = ['virginamerica','united'])
X_fit = count_vectorizer.fit_transform(clean_data).toarray()

In [18]:
X_fit.shape

(14404, 5000)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model = MultinomialNB()

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_fit,Y, test_size = 0.3)

In [22]:
model.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import classification_report

classification = classification_report(Y_test,y_pred)
print(classification)

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      2810
           1       0.62      0.47      0.53       885
           2       0.67      0.68      0.68       627

    accuracy                           0.77      4322
   macro avg       0.71      0.68      0.69      4322
weighted avg       0.76      0.77      0.77      4322

